# Lecture 2-2: Random numbers and Markov chain Monte Carlo

### Introduction

Random numbers can have many uses in programming and in physics. Here, we'll start to explore how random numbers can be used for a method of stochastic simulation called Markov chain Monte Carlo. Let's begin by first looking at how to generate random numbers.


### Generating random numbers

Python provides a standard library, `random`, for random number generation [here](https://docs.python.org/3/library/random.html). Numpy has a more comprehensive library [here](https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html). For the examples below, we'll use `numpy.random`. 

Random number generators in Python are not really random. They use values called **seeds** to determine the initial state of the random number generator. Given a specific seed as input, the output of a random number generator is totally deterministic. However, the numbers that appear are very close to random in a statistical sense. (For much more detail, see the related [Wikipedia page](https://en.wikipedia.org/wiki/Random_number_generation).)

In [1]:
import numpy as np
import numpy.random as rng
import seaborn as sns
import matplotlib.pyplot as plt

# First, let's define a new random number generator and set the seed

r = rng.RandomState(0)

# We can now use it to generate random numbers

n = r.rand()
print(n)

0.5488135039273248


In [2]:
# The sequence of random numbers generated by the same generator 
# with the same seed is always the same

print(np.array([n] + list(r.rand(5))))

r2 = rng.RandomState(0)
print(r2.rand(6))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411]
[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548  0.64589411]


### Using random numbers to "measure" the state of a stochastic system

As we just discussed, the energy of a particle with magnetic moment $\mathbf{\mu}$ in a magnetic field $\mathbf{B}$ is given by

$$ E = -\mathbf{\mu}\cdot \mathbf{B}\,.$$

Let's consider a statistical model of a particle in an external magnetic field. To keep things simple, we'll assume that the magnetic field is oriented in a fixed direction, and that the magnetic moment of the particle can be either aligned or anti-aligned with the magnetic field. In statistical physics models, these particles are often called **spins**.

Mathematically, we could describe the orientation of the spin with a variable $\sigma \in \{-1, 1\}$. Let's assume that the magnetic field is oriented in the positive direction. The energy of the spin is then

$$ E(\sigma) = -\epsilon \sigma\,, $$

where $\epsilon = \mu B$. If we maintain the system with the spin at a constant temperature $T$, then the Gibbs distribution for the spin states $\sigma$ is

$$ P(\sigma) = \frac{e^{-E(\sigma)/T}}{Z} \,.$$

Here we've chosen units such that Boltzmann's constant $k_B=1$. In other words, each time we observe the system, there's a probability $P(+1)$ to find the spin in the $+$ state, and a probability $P(-1)$ to find the spin in the $-$ state.

**If we observed the system 10 times, what would the average value of the spin be?**

To begin answering this question, let's write down the probabilities $P(+1)$ and $P(-1)$.

In [3]:
eps = 1 # magnitude of the energy epsilon = mu * B
T = 1   # temperature of the system

Z = np.exp(-eps/T) + np.exp(eps/T) # fill this in
p_plus = np.exp(eps/T) / Z # fill this in
p_minus = np.exp(-eps/T) / Z # fill this in

print('P(+) =', p_plus)
print('P(-) =', p_minus)

P(+) = 0.8807970779778824
P(-) = 0.11920292202211756


### Conditional expressions

Each time we measure the spin, we know the probability to find it in the plus or the minus state. If we want to use these probabilities to set the value of the spin each time we measure it, we can use a **conditional expression**. 

The most common conditional expressions are if/else statements, which work in a very intuitive way! Consider the example below.

In [4]:
def check_size(x):
    """ This function checks to see whether or not a number is larger than one."""
    if x>1:
        print('The number is larger than one')
    else:
        print('The number is not larger than one')
        
check_size(0)
check_size(2)

The number is not larger than one
The number is larger than one


How does this work? Python takes everything in between the `if` and the `:` and evaluates it as a **boolean** (True/False) statement. If the expression evaluates to `True`, then Python executes the next block of code. If it evaluates to `False`, then Python executes the block of code that follows `else:`.

Here are a list of some common comparison operators that you can use to generate True/False expressions:
- `>` greater than  
- `<` less than  
- `==` equal to (**note that there are two equal signs!**)  
- `!=` not equal to  
- `>=` greater than or equal to  
- `<=` less than or equal to  

These expressions can also be modified in multiple ways, but we won't go into the details at the moment. Let's practice with a few examples in the space below.

In [10]:
3==3

True

### Bringing random numbers and conditional expressions together

Behold.

In [12]:
x = r.rand()
if x<0.5:
    print('%lf is <0.5' % x)
else:
    print('%lf is >=0.5' % x)

0.891773 is >=0.5


We can use conditional expressions and probabilities to **simulate** a number of measurements of the spin, allowing us to estimate the magnetization. For example, let's imagine that we measured the spin 10 times. To find the average magnetization, we could use the following block of code.

We can also consider what happens as we increase the number of measurements.

In [21]:
n_measure = 10
m_avg = 0
for i in range(n_measure):
    if r.rand()<p_plus:
        m_avg = m_avg + 1
    else:
        m_avg = m_avg - 1

m_avg = m_avg / n_measure # normalize by dividing by number of measurements
print('The average magnetization is', m_avg)

The average magnetization is 0.8


### The Ising model

As a simple model of a ferromagnetic material, we can consider a set of spins arranged on a lattice. The energy of a pair of spins will be lower if they are aligned in the same direction. To make our analysis simple, we'll assume that only **nearest neighbor** spins are coupled together. Spins that aren't nearest neighbors could also interact, but these interactions will be much weaker due to the greater distance between them.

Mathematically, we can write the energy of a configuration $\underline{\sigma}=\{\sigma_1, \sigma_2, \ldots, \sigma_N\}$ of spins $\sigma_i\in\{-1,1\}$ in a one-dimensional lattice as

$$ 
E(\underline{\sigma}) = -\sum_{i=1}^{N-1} J \sigma_i\,\sigma_{i+1}\,.
$$

The coupling $J$ tells us how strong the interaction is between two neighboring spins.

To start, let's use a list to keep track of all the Ising spins.

In [22]:
N = 20      # number of spins in our system
spins = []  # the list of spins

# Fill in code here to initialize the list of spins
# Can we make the spins alternate + - + - ...?
for i in range(N):
  if i%2==0:
    spins.append(1)
  else:
    spins.append(-1)

print(spins)

[1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1]


Now, let's write a function to compute the energy of a configuration. For the moment, let's set $J=1$ for simplicity. The input to the function should be the configuration of spins, and the output should be the energy of the configuration. **Note: we need to be careful about the boundaries.** 

In [23]:
def compute_energy(spins):
    """ This function computes the energy of a spin configuration assuming J=1. """
    
    energy = 0
    # Fill in code here to add up the energy
    for i in range(len(spins)-1):
      energy = energy - spins[i]*spins[i+1]

    return energy

# Let's check to make sure that the energy function works

print('E( -1 -1 ) =', compute_energy([-1, -1]), '\t(expected -1)')
print('E( +1 +1 ) =', compute_energy([ 1,  1]), '\t(expected -1)')
print('E( -1 +1 ) = ', compute_energy([-1,  1]), '\t(expected  1)')

E( -1 -1 ) = -1 	(expected -1)
E( +1 +1 ) = -1 	(expected -1)
E( -1 +1 ) =  1 	(expected  1)
